In [2]:
a = range(1, 10)
for i in a:
    print(i)

1
2
3
4
5
6
7
8
9


In [3]:
import os, sys

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
np.random.seed(10)

# Configure matplotlib to use LaTeX for text rendering and set font size
plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 22})

def create_pfm( s, a, dt, bins ):
    '''compute the joint PMF of variables
        
        pfm = create_pfm( s, a, dt )
    Parameters
        s:      [np.ndarray]
            temporal evolution of target variable [size (Nt,)]
        a:      [ tuple ]
            each element must be an np.ndarray of size (Nt,)
            with the temporal evolution of agent variables
        dt:     [int]
            time lag in number of time steps
        bins:     [int]
            number of bins (states) per dimension
    Returns
        pfm     [np.ndarray]
            Mass probability function ( size ( bins, ..., bins ), dim = 1 + len(a) )
    
    '''
    V = np.vstack([ s[dt:], [ a[i][:-dt] for i in range(len(a)) ] ]).T
    
    # Histogram computes the bins by equally splitting the interval max(var)-min(var)
    h, _ = np.histogramdd( V, bins=bins )
    return h/h.sum()

In [4]:
N = int( 1e7 )
dt = 1
nbins = 2
    
q1 = np.random.rand( N ).round().astype(int)
q2 = np.random.rand( N ).round().astype(int)
# Example 1: Duplicated input
target = np.roll( q1, dt)
agents = ( q1, q1 )
nvars = 2

hist = create_pfm(target, agents, dt, nbins)
print(hist)

[[[0.49981725 0.        ]
  [0.         0.        ]]

 [[0.         0.        ]
  [0.         0.50018275]]]


In [5]:
hist.ndim

3

In [6]:
hist.shape

(2, 2, 2)

In [ ]:
hist

In [ ]:
hist.n